In [2]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define sectors and representative ETFs (Exchange Traded Funds) for each sector
sector_etfs = {
    "Technology": "XLK",
    "Finance": "XLF",
    "Healthcare": "XLV",
    "Energy": "XLE",
    "Consumer Discretionary": "XLY",
    "Consumer Staples": "XLP",
    "Utilities": "XLU",
    "Materials": "XLB",
    "Real Estate": "XLRE",
    "Industrials": "XLI"
}

# Download historical data for the ETFs
sector_data = yf.download(list(sector_etfs.values()), period="1y")["Adj Close"]

# Normalize the price data (set the first value to 1 for comparison)
normalized_data = sector_data / sector_data.iloc[0]

# Function to calculate MACD
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    ema_short = data.ewm(span=short_window, adjust=False).mean()
    ema_long = data.ewm(span=long_window, adjust=False).mean()
    macd = ema_short - ema_long
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal

# Plot 1: Normalized Sector Indices using Plotly
fig_normalized = go.Figure()

for sector, etf in sector_etfs.items():
    fig_normalized.add_trace(go.Scatter(
        x=normalized_data.index, 
        y=normalized_data[etf], 
        mode='lines',
        name=f"{sector} ({etf})",
        hovertemplate=f"<b>{sector} ({etf})</b><br>Date: %{{x}}<br>Price: %{{y:.2f}}<extra></extra>"
    ))

fig_normalized.update_layout(
    title="Normalized Sector Indices (Last 1 Year)",
    xaxis_title="Date",
    yaxis_title="Normalized Price",
    hovermode="x unified"
)

# Display the normalized sector chart
fig_normalized.show()

# Plot 2: MACD and Signal Lines using Plotly subplots
fig_macd = make_subplots(rows=len(sector_etfs), cols=1, shared_xaxes=True, subplot_titles=[f"{sector} ({etf})" for sector, etf in sector_etfs.items()])

for i, (sector, etf) in enumerate(sector_etfs.items(), start=1):
    data = sector_data[etf]
    macd, signal = calculate_macd(data)
    
    # Add MACD line
    fig_macd.add_trace(go.Scatter(
        x=data.index, 
        y=macd, 
        mode='lines',
        name=f"{sector} MACD ({etf})",
        hovertemplate=f"<b>{sector} ({etf})</b><br>Date: %{{x}}<br>MACD: %{{y:.2f}}<extra></extra>",
        line=dict(color='green')
    ), row=i, col=1)
    
    # Add Signal line
    fig_macd.add_trace(go.Scatter(
        x=data.index, 
        y=signal, 
        mode='lines',
        name=f"{sector} Signal ({etf})",
        hovertemplate=f"<b>{sector} ({etf})</b><br>Date: %{{x}}<br>Signal: %{{y:.2f}}<extra></extra>",
        line=dict(color='red')
    ), row=i, col=1)

# Update layout for the MACD plot
fig_macd.update_layout(
    height=1000,  # Adjust height as needed
    title_text="Sector MACD Analysis",
    hovermode="x unified"
)

# Display the MACD chart
fig_macd.show()


[*********************100%***********************]  10 of 10 completed


In [3]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from scipy.signal import argrelextrema
import numpy as np

# Define sectors and representative ETFs (Exchange Traded Funds) for each sector
sector_etfs = {
    "Technology": "XLK",
    "Finance": "XLF",
    "Healthcare": "XLV",
    "Energy": "XLE",
    "Consumer Discretionary": "XLY",
    "Consumer Staples": "XLP",
    "Utilities": "XLU",
    "Materials": "XLB",
    "Real Estate": "XLRE",
    "Industrials": "XLI"
}

# Download historical data for the ETFs
sector_data = yf.download(list(sector_etfs.values()), period="1y")["Adj Close"]

# Normalize the price data (set the first value to 1 for comparison)
normalized_data = sector_data / sector_data.iloc[0]

# Function to calculate a simple moving average (SMA)
def moving_average(data, window):
    return data.rolling(window=window).mean()

# Function to find local minima and maxima
def find_extrema(data, order=5):
    local_min = argrelextrema(data.values, np.less, order=order)[0]
    local_max = argrelextrema(data.values, np.greater, order=order)[0]
    return local_min, local_max

# Create individual charts for each sector
for sector, etf in sector_etfs.items():
    # Get the data for the current sector
    data = normalized_data[etf]
    
    # Find local minima (dips) and maxima (tops)
    local_min, local_max = find_extrema(data)
    
    # Create a figure for the current sector
    fig = go.Figure()

    # Plot the main price chart
    fig.add_trace(go.Scatter(
        x=data.index,
        y=data,
        mode='lines',
        name=f"{sector} ({etf})",
        hovertemplate=f"<b>{sector} ({etf})</b><br>Date: %{{x}}<br>Price: %{{y:.2f}}<extra></extra>"
    ))

    # Add moving average
    fig.add_trace(go.Scatter(
        x=data.index,
        y=moving_average(data, window=50),
        mode='lines',
        name=f"{sector} 50-day MA ({etf})",
        line=dict(dash='dot'),
        hovertemplate=f"<b>{sector} 50-day MA ({etf})</b><br>Date: %{{x}}<br>Price: %{{y:.2f}}<extra></extra>"
    ))

    # Mark the dips (local minima)
    fig.add_trace(go.Scatter(
        x=data.index[local_min],
        y=data.iloc[local_min],
        mode='markers',
        name=f"{sector} Dips",
        marker=dict(color='red', size=14, symbol='triangle-down'),
        hovertemplate=f"<b>Dip</b><br>Date: %{{x}}<br>Price: %{{y:.2f}}<extra></extra>"
    ))

    # Mark the tops (local maxima)
    fig.add_trace(go.Scatter(
        x=data.index[local_max],
        y=data.iloc[local_max],
        mode='markers',
        name=f"{sector} Tops",
        marker=dict(color='green', size=14, symbol='triangle-up'),
        hovertemplate=f"<b>Top</b><br>Date: %{{x}}<br>Price: %{{y:.2f}}<extra></extra>"
    ))

    # Layout options to enhance readability
    fig.update_layout(
        title=f"{sector} ({etf}) Price with Dips and Tops",
        xaxis_title="Date",
        yaxis_title="Normalized Price",
        hovermode="x unified",
        showlegend=True
    )

    # Display the chart for the current sector
    fig.show()


[*********************100%***********************]  10 of 10 completed


In [4]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from scipy.signal import argrelextrema
import numpy as np

# Define sectors and representative ETFs (Exchange Traded Funds) for each sector
sector_etfs = {
    "Technology": "XLK",
    "Finance": "XLF",
    "Healthcare": "XLV",
    "Energy": "XLE",
    "Consumer Discretionary": "XLY",
    "Consumer Staples": "XLP",
    "Utilities": "XLU",
    "Materials": "XLB",
    "Real Estate": "XLRE",
    "Industrials": "XLI"
}

# Download historical data for the ETFs
sector_data = yf.download(list(sector_etfs.values()), period="1y")["Adj Close"]

# Normalize the price data (set the first value to 1 for comparison)
normalized_data = sector_data / sector_data.iloc[0]

# Function to find local minima and maxima
def find_extrema(data, order=5):
    local_min = argrelextrema(data.values, np.less, order=order)[0]
    local_max = argrelextrema(data.values, np.greater, order=order)[0]
    return local_min, local_max

# Function to segment data based on upward or downward trend
def segment_trend(data):
    segments = []
    for i in range(1, len(data)):
        color = 'green' if data[i] > data[i-1] else 'red'
        segments.append((data.index[i-1:i+1], data[i-1:i+1], color))
    return segments

# Store dips and peaks across all sectors to detect opportunity zones
sector_min_dates = []

# Create a figure to plot all sectors
fig = go.Figure()

for sector, etf in sector_etfs.items():
    data = normalized_data[etf]

    # Find local minima (dips) and maxima (tops)
    local_min, local_max = find_extrema(data)
    
    # Save local minima dates for all sectors to detect multi-sector dips later
    sector_min_dates.extend(data.index[local_min])
    
    # Break the line into segments and color them based on trend (up or down)
    segments = segment_trend(data)
    for x_values, y_values, color in segments:
        fig.add_trace(go.Scatter(
            x=x_values,
            y=y_values,
            mode='lines',
            line=dict(color=color),
            name=f"{sector} ({etf})",
            showlegend=False,
            hovertemplate=f"<b>{sector} ({etf})</b><br>Date: %{{x}}<br>Price: %{{y:.2f}}<extra></extra>"
        ))

# Detect multi-sector dips (more than 3 sectors moving down)
sector_min_dates = pd.Series(sector_min_dates).value_counts()
multi_sector_dips = sector_min_dates[sector_min_dates >= 3].index

# Add vertical lines for multi-sector dips (sell-off opportunities)
for dip_date in multi_sector_dips:
    fig.add_trace(go.Scatter(
        x=[dip_date, dip_date],
        y=[0, 1.5],
        mode='lines',
        line=dict(color='blue', width=2, dash='dash'),
        name='Opportunity Zone',
        hovertemplate=f"<b>Multi-Sector Opportunity</b><br>Date: {dip_date}<extra></extra>",
        showlegend=False
    ))

# Layout options to enhance readability
fig.update_layout(
    title="Normalized Sector Indices with Trend Colors and Opportunity Zones",
    xaxis_title="Date",
    yaxis_title="Normalized Price",
    hovermode="x unified",
    legend_title="Sector",
    showlegend=False
)

# Show the chart
fig.show()


[*********************100%***********************]  10 of 10 completed


In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px

# Define sectors and representative ETFs (Exchange Traded Funds) for each sector
sector_etfs = {
    "Technology": "XLK",
    "Finance": "XLF",
    "Healthcare": "XLV",
    "Energy": "XLE",
    "Consumer Discretionary": "XLY",
    "Consumer Staples": "XLP",
    "Utilities": "XLU",
    "Materials": "XLB",
    "Real Estate": "XLRE",
    "Industrials": "XLI"
}

# Download historical data for the ETFs (last 6 months)
sector_data = yf.download(list(sector_etfs.values()), period="6mo")["Adj Close"]

# Function to calculate RSI
def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Calculate RSI for each sector
rsi_data = sector_data.apply(calculate_rsi)

# Determine overbought and oversold sectors based on RSI
overbought_threshold = 70
oversold_threshold = 30

recommendations = []

for sector, etf in sector_etfs.items():
    latest_rsi = rsi_data[etf].iloc[-1]  # Get the latest RSI value for each sector
    if latest_rsi > overbought_threshold:
        recommendations.append({"Sector": sector, "Recommendation": "Sell (Overbought)", "RSI": latest_rsi})
    elif latest_rsi < oversold_threshold:
        recommendations.append({"Sector": sector, "Recommendation": "Buy (Oversold)", "RSI": latest_rsi})
    else:
        recommendations.append({"Sector": sector, "Recommendation": "Hold (Neutral)", "RSI": latest_rsi})

# Convert recommendations to DataFrame for visualization
recommendations_df = pd.DataFrame(recommendations)

# Create a bar plot for the recommendations
fig = px.bar(recommendations_df, x="Sector", y="RSI", color="Recommendation", 
             title="Sector Rotation Recommendations Based on RSI",
             labels={"RSI": "RSI (Relative Strength Index)"}, 
             color_discrete_map={
                 "Sell (Overbought)": "red", 
                 "Buy (Oversold)": "green", 
                 "Hold (Neutral)": "gray"})

# Update layout for readability
fig.update_layout(xaxis_title="Sectors", yaxis_title="RSI", 
                  hovermode="x unified", 
                  showlegend=True)

# Show the recommendations bar chart
fig.show()


/Users/kwaltman/mambaforge/envs/awesome/lib/python3.9/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


[*********************100%***********************]  10 of 10 completed


In [3]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define sectors and representative ETFs (Exchange Traded Funds) for each sector
sector_etfs = {
    "Technology": "XLK",
    "Finance": "XLF",
    "Healthcare": "XLV",
    "Energy": "XLE",
    "Consumer Discretionary": "XLY",
    "Consumer Staples": "XLP",
    "Utilities": "XLU",
    "Materials": "XLB",
    "Real Estate": "XLRE",
    "Industrials": "XLI"
}

# Download historical data for the ETFs
sector_data = yf.download(list(sector_etfs.values()), period="1y")["Adj Close"]

# Calculate the return for each ETF over the last year
returns = (sector_data.iloc[-1] / sector_data.iloc[0] - 1).sort_values(ascending=False)

# Create a ranking DataFrame
ranked_df = pd.DataFrame({
    "Sector": [sector for sector, etf in sector_etfs.items()],
    "ETF": [etf for sector, etf in sector_etfs.items()],
    "Return": [returns[etf] for sector, etf in sector_etfs.items()]
}).sort_values(by="Return", ascending=False).reset_index(drop=True)

# Add a Rank column
ranked_df["Rank"] = ranked_df.index + 1

# Print the ranking DataFrame
print(ranked_df)

# Sort sector ETFs by returns (highest return to lowest) for plotting
sorted_sector_etfs = {row["Sector"]: row["ETF"] for _, row in ranked_df.iterrows()}

# Normalize the price data for visualization
normalized_data = sector_data / sector_data.iloc[0]

# Function to calculate MACD
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    ema_short = data.ewm(span=short_window, adjust=False).mean()
    ema_long = data.ewm(span=long_window, adjust=False).mean()
    macd = ema_short - ema_long
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal

# Plot 1: Normalized Sector Indices using Plotly
fig_normalized = go.Figure()

for sector, etf in sorted_sector_etfs.items():
    fig_normalized.add_trace(go.Scatter(
        x=normalized_data.index, 
        y=normalized_data[etf], 
        mode='lines',
        name=f"{sector} ({etf})",
        hovertemplate=f"<b>{sector} ({etf})</b><br>Date: %{{x}}<br>Price: %{{y:.2f}}<extra></extra>"
    ))

fig_normalized.update_layout(
    title="Normalized Sector Indices (Last 1 Year) - Sorted by Return",
    xaxis_title="Date",
    yaxis_title="Normalized Price",
    hovermode="closest"
)

# Display the normalized sector chart
fig_normalized.show()

# Plot 2: MACD and Signal Lines using Plotly subplots, sorted by highest return
fig_macd = make_subplots(rows=len(sorted_sector_etfs), cols=1, shared_xaxes=True, subplot_titles=[f"{sector} ({etf})" for sector, etf in sorted_sector_etfs.items()])

for i, (sector, etf) in enumerate(sorted_sector_etfs.items(), start=1):
    data = sector_data[etf]
    macd, signal = calculate_macd(data)
    
    # Add MACD line
    fig_macd.add_trace(go.Scatter(
        x=data.index, 
        y=macd, 
        mode='lines',
        name=f"{sector} MACD ({etf})",
        hovertemplate=f"<b>{sector} ({etf})</b><br>Date: %{{x}}<br>MACD: %{{y:.2f}}<extra></extra>",
        line=dict(color='green')
    ), row=i, col=1)
    
    # Add Signal line
    fig_macd.add_trace(go.Scatter(
        x=data.index, 
        y=signal, 
        mode='lines',
        name=f"{sector} Signal ({etf})",
        hovertemplate=f"<b>{sector} ({etf})</b><br>Date: %{{x}}<br>Signal: %{{y:.2f}}<extra></extra>",
        line=dict(color='red')
    ), row=i, col=1)

# Update layout for the MACD plot
fig_macd.update_layout(
    height=1000,  # Adjust height as needed
    title_text="Sector MACD Analysis - Sorted by Return",
    hovermode="closest"
)

# Display the MACD chart
fig_macd.show()


[*********************100%***********************]  10 of 10 completed
                   Sector   ETF    Return  Rank
0                 Finance   XLF  0.496323     1
1             Industrials   XLI  0.411806     2
2  Consumer Discretionary   XLY  0.359764     3
3               Utilities   XLU  0.355792     4
4              Technology   XLK  0.335702     5
5             Real Estate  XLRE  0.314371     6
6               Materials   XLB  0.224206     7
7        Consumer Staples   XLP  0.199000     8
8              Healthcare   XLV  0.175922     9
9                  Energy   XLE  0.165789    10
